# 1. 환경설정

In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [2]:
with open('df_sector_UTD.pickle','rb') as f:
    df_sector_UTD = pickle.load(f)
with open('certificated_stock_dic.pickle','rb') as f:
    certificated_stock_dic = pickle.load(f)
with open('X_scaler_dic.pickle','rb' ) as f:
    X_scaler_dic = pickle.load(f)
with open('y_scaler_dic.pickle','rb' ) as f:
    y_scaler_dic = pickle.load(f)   

In [3]:
for sector_name, sector_dic in df_sector_UTD.items():
    for stock_code, df in sector_dic.items():
        df.set_index('일자',inplace=True)

# 2. 스케일링

In [5]:
from sklearn.preprocessing import MinMaxScaler
scale_cols = ['거래량','시가','고가', '저가']

X_UTD_scaled_dic = {}
for sector_name, stock_list in certificated_stock_dic.items():
    X_UTD_scaled_dic[sector_name] = {}
    for stock_code in stock_list:
        X_scaler = X_scaler_dic[sector_name][stock_code]
        X_UTD_scaled = X_scaler.transform(df_sector_UTD[sector_name][stock_code][scale_cols])
        X_UTD_scaled_dic[sector_name][stock_code] = pd.DataFrame(X_UTD_scaled,index = df_sector_UTD[sector_name][stock_code].index,columns=scale_cols)
        

In [6]:
X_UTD_scaled_dic

{'Food': {'271560':                  거래량        시가        고가        저가
  일자                                                
  2017-07-07  1.000000  0.107053  0.110583  0.000000
  2017-07-10  0.511056  0.046599  0.090369  0.065594
  2017-07-11  0.248741  0.118388  0.090369  0.086634
  2017-07-12  0.133379  0.064232  0.049941  0.053218
  2017-07-13  0.120664  0.060453  0.042806  0.063119
  ...              ...       ...       ...       ...
  2022-08-10  0.078617  0.351385  0.346017  0.368812
  2022-08-11  0.051524  0.351385  0.322235  0.362624
  2022-08-12  0.026819  0.338791  0.316290  0.337871
  2022-08-16  0.036159  0.332494  0.316290  0.344059
  2022-08-17  0.061379  0.338791  0.340071  0.362624
  
  [1257 rows x 4 columns],
  '000080':                  거래량        시가        고가        저가
  일자                                                
  2010-06-08  0.007946  0.683673  0.705521  0.694672
  2010-06-09  0.007270  0.704082  0.703476  0.706967
  2010-06-10  0.013428  0.700000  0.70756

In [7]:
df_sector_UTD

{'Food': {'097950':               거래량      시가      고가      저가      종가
  일자                                               
  2010-06-08  50727  211000  213500  209500  211000
  2010-06-09  52375  212000  218000  210500  214000
  2010-06-10  61195  214500  219500  213500  219000
  2010-06-11  89684  217000  220000  211500  214500
  2010-06-14  41941  216000  220000  215500  219000
  ...           ...     ...     ...     ...     ...
  2022-08-10  66726  426500  428500  414500  423500
  2022-08-11  66420  425000  425000  417000  421500
  2022-08-12  43606  419000  422000  414000  416500
  2022-08-16  31340  418000  420500  416500  418500
  2022-08-17  30523  419500  423000  413500  417000
  
  [3010 rows x 5 columns],
  '271560':                 거래량      시가      고가      저가      종가
  일자                                                 
  2017-07-07  1619864   87100   89700   75700   82300
  2017-07-10   840451   82300   88000   81000   88000
  2017-07-11   422303   88000   88000   82700   82

In [8]:
#윈도우 사이즈
WINDOW_SIZE = 20
# X에 대한 윈도우 함수 설정값
X_size = WINDOW_SIZE
X_shift = 1
X_stride = 1
# y에 대한 윈도우 함수 설정값
y_size = 1
y_shift = 1
y_stride = 1

batch_size = 32

# 2. 모델 불러오기

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
with open('filename_dic.pickle', 'rb') as f:
    filename_dic = pickle.load(f)

In [11]:
model_dic ={}
for sector_name, stock_list in certificated_stock_dic.items():
    model_dic[sector_name] = {}
    for stock_code in stock_list:
        model_dic[sector_name][stock_code] = Sequential([
            # 1차원 feature map 생성
            Conv1D(filters=32, kernel_size=5,
                   padding="causal",
                   activation="relu",
                   #feature의 개수 3개
                   input_shape=[WINDOW_SIZE, 4]),
            # LSTM
            LSTM(16, activation='tanh'),
            Dense(16, activation="relu"),
            Dense(1),
        ])

# 리눅스 환경으로 변경

In [12]:
for_linux = {}
for sector_name, sector_name_dic in certificated_stock_dic.items():
    for_linux[sector_name] = {}
    for stock_code in sector_name_dic:
        linux_filename = filename_dic[sector_name][stock_code].replace('\\','/')
        for_linux[sector_name][stock_code] = linux_filename

In [13]:
for sector_name, filename_sector in for_linux.items():
    for stock_code, filename in filename_sector.items():
        model_dic[sector_name][stock_code].load_weights(filename)

# 검증된 항목에서 과거 5일 기준 tomorrow예측하기

In [14]:
from datetime import date,datetime, timedelta
today = date.today()
tomorrow = today+timedelta(days=1, hours = 9)
tomorrow = tomorrow.isoformat()
tomorrow = tomorrow.replace('-','')[2:]
set_d = tomorrow
set_d

'220818'

In [15]:
pred_dic = {}
for sector_name, sector_model in model_dic.items():
    pred_dic[sector_name] = {}
    # 과거5일통해 내일 주가 예측
    for stock_code, model in sector_model.items():
        tomorrow = tf.constant(np.expand_dims(np.array(X_UTD_scaled_dic[sector_name][stock_code].tail(20)[:20]),axis=0))
        today = tf.constant(np.expand_dims(np.array(X_UTD_scaled_dic[sector_name][stock_code].tail(21)[:20]),axis=0))
        pred_today = model.predict(today)
        pred_tomorrow = model.predict(tomorrow)
        y_scaler = y_scaler_dic[sector_name][stock_code]
        origin_pred_tomorrow = y_scaler.inverse_transform(pred_tomorrow)
        origin_pred_tomorrow = np.around(origin_pred_tomorrow,0)
        origin_pred_tomorrow = int(origin_pred_tomorrow)

        
        #상향 : 2, 하향 : 1, 횡보 : 0
        if pred_tomorrow > pred_today:
            print(f'{stock_code}은 상향를 예상합니다.')
            pred_dic[sector_name][stock_code] = (2,origin_pred_tomorrow)
        elif pred_tomorrow < pred_today:
            print(f'{stock_code}은 하향를 예상합니다.')
            pred_dic[sector_name][stock_code] = (1,origin_pred_tomorrow)
        elif pred_tomorrow == pred_today:
            print(f'{stock_code}은 횡보를 예상합니다.')
            pred_dic[sector_name][stock_code] = (0,origin_pred_tomorrow)

271560은 상향를 예상합니다.
000080은 하향를 예상합니다.
004370은 하향를 예상합니다.
005300은 하향를 예상합니다.
093050은 상향를 예상합니다.
020000은 하향를 예상합니다.
105630은 하향를 예상합니다.
096770은 상향를 예상합니다.
010950은 상향를 예상합니다.
051900은 하향를 예상합니다.
090430은 상향를 예상합니다.
207940은 상향를 예상합니다.
068270은 상향를 예상합니다.
128940은 하향를 예상합니다.


003410은 상향를 예상합니다.
010780은 상향를 예상합니다.
005490은 상향를 예상합니다.
010130은 상향를 예상합니다.
004020은 상향를 예상합니다.
016380은 상향를 예상합니다.
001230은 상향를 예상합니다.
034020은 상향를 예상합니다.
018880은 하향를 예상합니다.
241560은 상향를 예상합니다.
005930은 하향를 예상합니다.


000660은 상향를 예상합니다.
066570은 상향를 예상합니다.
000720은 상향를 예상합니다.
006360은 상향를 예상합니다.
047040은 상향를 예상합니다.
051600은 상향를 예상합니다.
011200은 하향를 예상합니다.
003490은 상향를 예상합니다.
086280은 상향를 예상합니다.
180640은 상향를 예상합니다.
028670은 상향를 예상합니다.


028260은 상향를 예상합니다.
023530은 상향를 예상합니다.
282330은 하향를 예상합니다.
139480은 하향를 예상합니다.
004170은 하향를 예상합니다.
015760은 상향를 예상합니다.
036460은 상향를 예상합니다.
030200은 하향를 예상합니다.
032640은 하향를 예상합니다.
003550은 상향를 예상합니다.
000810은 하향를 예상합니다.


006800은 상향를 예상합니다.
005940은 상향를 예상합니다.
016360은 상향를 예상합니다.
008560은 상향를 예상합니다.
039490은 상향를 예상합니다.
032830은 상향를 예상합니다.
005830은 하향를 예상합니다.
001450은 하향를 예상합니다.
018260은 상향를 예상합니다.
036570은 하향를 예상합니다.
005380은 하향를 예상합니다.


000270은 상향를 예상합니다.
012330은 하향를 예상합니다.
033780은 상향를 예상합니다.
009150은 상향를 예상합니다.


In [16]:
origin_pred_tomorrow

136116

In [18]:
from datetime import date, timedelta,datetime
rinuxtimenow = datetime.now() #리눅스 현재 시각
tomorrow = rinuxtimenow+timedelta(days=1,hours=9) #kr 24시간 후
set_d = tomorrow.date()
set_d = set_d.isoformat()
set_d = set_d.replace('-','')[2:]

In [20]:
with open(f'{set_d}pred_dic.pickle','wb') as f:
    pickle.dump(pred_dic,f)